In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pickle
#from rembg import remove

In [ ]:
def get_directories(path):
    # Get list of all files and directories in the given path
    contents = os.listdir(path)
    
    # Filter out only directories
    directories = [content for content in contents if os.path.isdir(os.path.join(path, content))]
    
    return directories

In [ ]:
# print(directory_list)
# make sure to stick with rgb images
# make sure wwhen training the model, the pixel intensity is from 0 to 1 (normalize the images) (make it 0 mean)

In [ ]:
# create histogram of data distribution per class (both training and testing)
def create_data_hist(samples):
    # samples = [(class_name, count), ...]
    class_names = [item[0] for item in samples]
    num_samples = [item[1] for item in samples]
    
    for i, name in enumerate(class_names):
        print(name, num_samples[i])
    
    # Plot histogram
    plt.bar(class_names, num_samples, color='skyblue')
    plt.xlabel('Class Name')
    plt.ylabel('Number of Samples')
    plt.title('Data Distribution Histogram')
    plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility
    plt.tight_layout()  # Adjust layout to prevent clipping of labels
    plt.show()

In [ ]:
# Processing our dataset
# dataset_type = {'train', 'val'}
def create_dataset(dataset_type):
    directory_list = get_directories(f'./{dataset_type}')
    data = []
    CATEGORIES = directory_list
    DATADIR = f'./{dataset_type}/'
    samples = []
    classdict = {}
    # category number: category name

    for category in CATEGORIES:
        count = 0
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        classdict[class_num] = category
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                #remove background
                #img_array = remove(img_array)
                new_arr = np.moveaxis(img_array, -1, 0)
                data.append((new_arr, class_num, img))
                count += 1
            except Exception as e:
                print(f"Error: {e}")
                pass
        samples.append((category, count))

    # If the class is 'Tomato___healthy', keep 1000 instances, else keep 400 instances
    balanced_data = []
    classlens = [0,0,0,0,0,0,0,0,0,0]
    a = False
    for d in data:
        # d[2] has the string "_HL" in it, it is a healthy tomato leaf image
        if not a and d[2].find("_HL") != -1:
            a = True
            print("Health class:",d[1])
            print(classdict[d[1]])
        if d[1] == 2 and classlens[d[1]] < 1000:
            balanced_data.append(d)
            classlens[d[1]] += 1
        else:
            if classlens[d[1]] < 400:
                balanced_data.append(d)
                classlens[d[1]] += 1

    create_data_hist(samples)  # Display original data distribution
    create_data_hist([('Tomato___healthy', 1000)] + [('Anomaly', 400) for _ in range(len(CATEGORIES)-1)])  # Display balanced data distribution
    
    # Shuffle the balanced data
    random.shuffle(balanced_data)

    return (balanced_data, classdict)


In [ ]:
def save_data(data, name):
    # Dumping training data
    out = name + ".pickle"
    pickle_out = open(out, "wb")
    pickle.dump(data[0], pickle_out)
    pickle_out.close()

In [15]:
t_data = create_dataset('train')
train_data = t_data[0]

KeyboardInterrupt: 

In [ ]:
train_data[0][0].shape

In [ ]:
# plotting images
image = train_data[100][0]
blue_channel, green_channel, red_channel = [image[0], image[1], image[2]]

# grayscale image
new_img = np.moveaxis(image, 0, -1)
gray_image = cv2.cvtColor(new_img, cv2.COLOR_RGB2GRAY)


# Plot each channel separately
plt.figure(figsize=(10, 5))

plt.subplot(1, 4, 1)
plt.imshow(blue_channel, cmap='Blues')
plt.title('Blue Channel')
# plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(green_channel, cmap='Greens')
plt.title('Green Channel')
# plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(red_channel, cmap='Reds')
plt.title('Red Channel')
# plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(gray_image, cmap='gray')
plt.title('Grayscale Img')
# plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
v_data = create_dataset('val')
test_data = v_data[0]

In [ ]:
#save_data(t_data, "train_data")
#save_data(v_data, "val_data")

In [ ]:
def create_dataset_directory(data_and_classes,name):
    # Create base directory for the dataset
    base_dir = f'./new_dataset/{name}'
    os.makedirs(base_dir, exist_ok=True)
    data = data_and_classes[0]
    classes = data_and_classes[1]
    print ("classes", classes)
    # Iterate over the data, which is expected to be a list of tuples (image_array, label)
    for idx, (image_array, label, imgname) in enumerate(data):
        # Create directory for each class if it doesn't exist
        class_dir = os.path.join(base_dir, classes[label])
        os.makedirs(class_dir, exist_ok=True)
        
        # Define path for the image to be saved
        image_path = os.path.join(class_dir, imgname)
        
        # Save the image
        new_arr = np.moveaxis(image_array, 0, -1)
        cv2.imwrite(image_path, cv2.cvtColor(new_arr, cv2.COLOR_RGB2BGR))  # Convert back to BGR for saving


In [ ]:
create_dataset_directory(t_data, "train")
create_dataset_directory(v_data, "val")